# 1. 라이브러리 Import


In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━

In [2]:
# 필요한 라이브러리 임포트
from google.colab import drive
import pandas as pd
import numpy as np
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    AdamW,
    get_linear_schedule_with_warmup,
    EarlyStoppingCallback,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import random
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files
from huggingface_hub import notebook_login
from peft import prepare_model_for_kbit_training
from transformers import pipeline
import random
from google.colab import files

In [3]:
random.seed(42)

# 2. 데이터 전처리

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file_path = '/content/drive/MyDrive/generativeai/games.csv'

In [6]:
df = pd.read_csv(file_path, usecols=["Title", "Rating", "Genres", "Summary", "Reviews"])

In [7]:
df.head()

,Title,Rating,Genres,Summary,Reviews
0,Elden Ring,4.5,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o..."
1,Hades,4.3,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...
2,The Legend of Zelda: Breath of the Wild,4.4,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...
3,Undertale,4.2,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...
4,Hollow Knight,4.4,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with..."


In [8]:
df.drop_duplicates(['Title'])

,Title,Rating,Genres,Summary,Reviews
0,Elden Ring,4.5,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o..."
1,Hades,4.3,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...
2,The Legend of Zelda: Breath of the Wild,4.4,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...
3,Undertale,4.2,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...
4,Hollow Knight,4.4,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with..."
...,...,...,...,...,...
1507,Back to the Future: The Game,3.2,"['Adventure', 'Point-and-Click']",Back to the Future: The Game is one of Telltal...,['Very enjoyable game. The story adds onto the...
1508,Team Sonic Racing,2.9,"['Arcade', 'Racing']",Team Sonic Racing combines the best elements o...,"['jogo morto mas bom', 'not my cup of tea', ""C..."
1509,Dragon's Dogma,3.7,"['Brawler', 'RPG']","Set in a huge open world, Dragon’s Dogma: Dark...","['Underrated.', 'A grandes rasgos, es como un ..."
1510,Baldur's Gate 3,4.1,"['Adventure', 'RPG', 'Strategy', 'Tactical', '...","An ancient evil has returned to Baldur's Gate,...",['Bu türe bu oyunla girmeye çalışmak hataydı s...


In [9]:
df.isnull().sum()

,0
Title,0
Rating,13
Genres,0
Summary,1
Reviews,0


In [10]:
df['Rating'] = df['Rating'].fillna(df['Rating'].mean())

In [11]:
df.isnull().sum()

,0
Title,0
Rating,0
Genres,0
Summary,1
Reviews,0


In [12]:
df = df.dropna(subset = 'Summary')

In [13]:
df.isnull().sum()

,0
Title,0
Rating,0
Genres,0
Summary,0
Reviews,0


In [14]:
df["Combined"] = (
    "Title: " + df["Title"].astype(str) + " " + # Title도 추가해서 combine에 넣었어요~!
    "Ratings: " + df["Rating"].astype(str) + " " +  # Rating을 문자열로 변환하고 접두사 추가
    "Genres: "+ df["Genres"].astype(str) + " " +   # Genres를 문자열로 변환
    "\nDescription: " + df["Summary"] + " " +      # Description 앞에 줄 바꿈 추가
    df["Reviews"]                                  # Reviews 추가
)

# 특수문자 제거
df["Combined"] = df["Combined"].str.replace(r'[\[\]"\'\\]', '', regex=True)
df["Reviews"] = df["Reviews"].str.replace(r'[\[\]"\'\\]', '', regex=True)
df["Summary"] = df["Summary"].str.replace(r'[\[\]"\'\\]', '', regex=True)

# 공백 문제 처리
df["Combined"] = df["Combined"].str.replace(r'\s+', ' ', regex=True).str.strip()
df["Reviews"] = df["Reviews"].str.replace(r'\s+', ' ', regex=True).str.strip()
df["Summary"] = df["Summary"].str.replace(r'\s+', ' ', regex=True).str.strip()

In [15]:
df.head()

,Title,Rating,Genres,Summary,Reviews,Combined
0,Elden Ring,4.5,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...",The first playthrough of elden ring is one of ...,Title: Elden Ring Ratings: 4.5 Genres: Adventu...
1,Hades,4.3,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,convinced this is a roguelike for people who d...,"Title: Hades Ratings: 4.3 Genres: Adventure, B..."
2,The Legend of Zelda: Breath of the Wild,4.4,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,This game is the game (that is not CS:GO) that...,Title: The Legend of Zelda: Breath of the Wild...
3,Undertale,4.2,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",soundtrack is tied for #1 with nier automata. ...,Title: Undertale Ratings: 4.2 Genres: Adventur...
4,Hollow Knight,4.4,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"this games worldbuilding is incredible, with i...",Title: Hollow Knight Ratings: 4.4 Genres: Adve...


In [16]:
preprocessed_data = df.drop(columns=['Genres', 'Rating', 'Reviews', 'Summary'])

In [17]:
print(preprocessed_data)

                                        Title  \
0                                  Elden Ring   
1                                       Hades   
2     The Legend of Zelda: Breath of the Wild   
3                                   Undertale   
4                               Hollow Knight   
...                                       ...   
1507             Back to the Future: The Game   
1508                        Team Sonic Racing   
1509                           Dragon's Dogma   
1510                          Baldur's Gate 3   
1511                 The LEGO Movie Videogame   

                                               Combined  
0     Title: Elden Ring Ratings: 4.5 Genres: Adventu...  
1     Title: Hades Ratings: 4.3 Genres: Adventure, B...  
2     Title: The Legend of Zelda: Breath of the Wild...  
3     Title: Undertale Ratings: 4.2 Genres: Adventur...  
4     Title: Hollow Knight Ratings: 4.4 Genres: Adve...  
...                                                 ...  
1507 

In [18]:
df['Genres'] = df['Genres'].str.replace(r"['\[\]]", '', regex=True)

# 3. training data 구성

In [19]:
# TF-IDF 벡터화
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df["Combined"])

In [20]:
def generate_user_queries(row):
    import random


    genres = row['Genres']
    summaries = row['Summary']
    title = row['Title']
    rating = row['Rating']
    review = row['Reviews']
    templates = [
        # 장르 기반 쿼리
        f"I am looking for a {genres} game.",
        f"Can you recommend a {genres} game?",
        f"Suggest me a {genres} game.",
        f"I want to try a {genres} game.",
        f"What are the best {genres} games?",
        f"Are there any {genres} games you recommend?",
        f"I need a new {genres} game to play.",
        f"Can you find me a good {genres} game?",


        # 기타 표현
        f"Can you find me a fun {genres} game?",
        f"I want a new adventure in the {genres} genre.",
        f"Recommend a game that's exciting and {genres}.",
        f"I need a {genres} game to pass the time.",
        f"I'm looking for a {genres} game that's worth playing."
         "I’m looking for a game that is calm and peaceful. Any ideas?",
        "Can you recommend a game with an exciting storyline?",
        "What’s a good game with beautiful landscapes to explore?",
        "Are there any games with a mix of action and strategy?",
        "I need a game where I can make important choices. Suggestions?",
        "What are the best games for exploring mysterious worlds?",
        "Can you help me find a game with a strong and emotional narrative?",
        "What’s a good game that offers freedom to roam and explore?",
        "I’m looking for a game that feels like an epic adventure.",
        "What are some games with magical or fantasy themes?",
        "Can you suggest a game with great puzzles and challenges?",
        "I’m in the mood for a game that’s all about discovery and wonder.",
        "What’s a game that blends creativity with engaging gameplay?",
        "I want to play a game where the environment feels alive. Any ideas?",
        "What’s a good game that lets me build and customize things freely?",
        "I’m searching for a game that’s relaxing yet immersive.",
        "What are the best games that focus on exploration and story?",
        "Can you recommend a game where I can just sit back and enjoy the story?",
        "What’s a game that takes me on an unforgettable journey?",
        "I want a game that’s easy to pick up but hard to master. Any suggestions?",
        "What’s a great game for playing with friends online?",
        "Can you suggest a game that’s perfect for a rainy day?",
        "What are some fun games for casual players?",
        "I want to dive into a game with a rich and detailed world. Any ideas?",
        "What’s a good game that I can play for hours without getting bored?",
        "I need a game that’s both challenging and rewarding.",
        "Can you recommend a game that’s great for creative thinkers?",
        "What’s a game that offers a sense of wonder and discovery?",
        "Are there any games that let me create and customize my experience?",
        "I’m looking for a game that’s perfect for short play sessions.",
        "What’s a good game that combines action with great visuals?",
        "I want a game that keeps me on the edge of my seat. Suggestions?",
        "What’s a game that’s both exciting and relaxing at the same time?",
        "I need a game that’s perfect for playing after work. Any ideas?",
        "Can you recommend a game that feels like a great escape?",
        # 기타 자주 묻는 질문들
        "What are the best free games available right now?",
        "Can you suggest a game I can play in short sessions?",
        "What are some of the top-rated games released this year?",
        "What’s a good game that’s perfect for multiplayer fun?",
        "Are there any games with a nostalgic feel worth playing?",
        "What’s a game that’s easy to get into but hard to stop playing?",
        "What are the most visually stunning games I should try?",
        "What’s a hidden gem that not many people know about?",
        "What’s a good game that balances fun and challenge perfectly?",
        "Can you suggest a game that feels like an emotional journey?",
        "What’s a recent game that everyone seems to be talking about?",
        "Are there any games that blend humor and adventure well?",
        "What’s a game that’s worth playing for its story alone?",
        "What’s a game with breathtaking visuals and an epic scale?",
        "What’s a game that’s both family-friendly and engaging?",
        "What’s a game that can make me feel like a hero?",
        "Can you suggest a game that feels rewarding and satisfying?",
        "What’s a game that offers a unique and unforgettable experience?",
        "What’s a great game for players who love to solve mysteries?",
        "What’s a good game that focuses on player creativity?",
    ]

    return random.choice(templates)


In [21]:
def create_training_data_with_cosine_similarity(df, tfidf_matrix, num_queries, top_k, threshold=0.1):
    """
    코사인 유사도가 threshold 이상일 때만 추천에 포함하며, 추천 결과가 없는 경우 데이터에서 제외.
    중복된 쿼리는 사용하지 않음.
    """
    training_data = []
    sampled_indices = random.sample(range(len(df)), num_queries)  # 중복 없는 샘플링
    seen_queries = set()  # 중복 쿼리 확인용 집합

    for idx in sampled_indices:
        random_row = df.iloc[idx]  # 데이터 선택
        user_query = generate_user_queries(random_row)  # 사용자 쿼리 생성

        # 중복 쿼리 제거
        if user_query in seen_queries:
            continue
        seen_queries.add(user_query)

        user_vector = tfidf.transform([user_query])  # 사용자 쿼리 벡터화
        cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()  # 코사인 유사도 계산
        top_indices = cosine_similarities.argsort()[::-1]  # 유사도가 높은 순서대로 정렬

        recommendations = []
        seen_titles = set()  # 중복 방지를 위한 집합

        for idx in top_indices:
            if len(recommendations) >= top_k:  # 최대 top_k개의 추천
                break

            # 유사도 조건 추가
            if cosine_similarities[idx] < threshold:  # 유사도가 threshold보다 낮으면 무시
                continue

            game_title = df.iloc[idx]['Title']
            if game_title in seen_titles:  # 중복된 게임은 제외
                continue

            game_summary = df.iloc[idx]['Combined']
            recommendations.append(f"Game: {game_title}\n {game_summary}")
            seen_titles.add(game_title)

        # 추천 결과가 없는 경우 데이터 추가하지 않음
        if recommendations:
            output_text = "\n\n".join(recommendations)
            training_data.append({
                "Text": f"input_text: {user_query} output_text: {output_text}"
            })

    return pd.DataFrame(training_data)


In [ ]:
del training_data

In [22]:
training_data = create_training_data_with_cosine_similarity(df, tfidf_matrix, num_queries=1500, top_k=10,threshold=0.1)

In [23]:
print(training_data)

                                                  Text
0    input_text: What’s a good game that combines a...
1    input_text: Are there any games with a mix of ...
2    input_text: What’s a game that’s easy to get i...
3    input_text: I need a game that’s perfect for p...
4    input_text: What’s a hidden gem that not many ...
..                                                 ...
241  input_text: I need a RPG game to pass the time...
242  input_text: I need a new Adventure, Tactical g...
243  input_text: Are there any Adventure, Shooter g...
244  input_text: Suggest me a Adventure, Strategy g...
245  input_text: I am looking for a Racing game. ou...

[246 rows x 1 columns]


In [ ]:
# Save the training data as a CSV file
file_path = "training_data.csv"
training_data.to_csv(file_path, index=False)

# Provide file for download
from google.colab import files
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
print(preprocessed_data["Combined"])

0       Title: Elden Ring Ratings: 4.5 Genres: Adventu...
1       Title: Hades Ratings: 4.3 Genres: Adventure, B...
2       Title: The Legend of Zelda: Breath of the Wild...
3       Title: Undertale Ratings: 4.2 Genres: Adventur...
4       Title: Hollow Knight Ratings: 4.4 Genres: Adve...
                              ...                        
1507    Title: Back to the Future: The Game Ratings: 3...
1508    Title: Team Sonic Racing Ratings: 2.9 Genres: ...
1509    Title: Dragons Dogma Ratings: 3.7 Genres: Braw...
1510    Title: Baldurs Gate 3 Ratings: 4.1 Genres: Adv...
1511    Title: The LEGO Movie Videogame Ratings: 2.8 G...
Name: Combined, Length: 1511, dtype: object


In [25]:
combined_texts = preprocessed_data['Combined'].dropna().rename('Text')
training_data = pd.concat([training_data, combined_texts.to_frame()], ignore_index=True)

In [26]:
print(training_data)

                                                   Text
0     input_text: What’s a good game that combines a...
1     input_text: Are there any games with a mix of ...
2     input_text: What’s a game that’s easy to get i...
3     input_text: I need a game that’s perfect for p...
4     input_text: What’s a hidden gem that not many ...
...                                                 ...
1752  Title: Back to the Future: The Game Ratings: 3...
1753  Title: Team Sonic Racing Ratings: 2.9 Genres: ...
1754  Title: Dragons Dogma Ratings: 3.7 Genres: Braw...
1755  Title: Baldurs Gate 3 Ratings: 4.1 Genres: Adv...
1756  Title: The LEGO Movie Videogame Ratings: 2.8 G...

[1757 rows x 1 columns]


In [1]:
# Save the training data as a CSV file
file_path = "training_data.csv"
training_data.to_csv(file_path, index=False)

# Provide file for download
from google.colab import files
files.download(file_path)


NameError: name 'training_data' is not defined

In [27]:
dataset = Dataset.from_pandas(training_data)
dataset = dataset.shuffle(seed = 42)
dataset = dataset.train_test_split(test_size=0.2)

In [28]:
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [ ]:
print(train_dataset)

Dataset({
    features: ['Text'],
    num_rows: 1420
})


In [ ]:
print(eval_dataset)

Dataset({
    features: ['Text'],
    num_rows: 356
})


In [ ]:
print(type(train_dataset["Text"]))

<class 'list'>


# 4. 모델 fine-tuning

In [30]:
notebook_login()

In [31]:
# 모델과 토크나이저 로드
from peft import prepare_model_for_kbit_training

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)
base_model = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="balanced_low_0",
    offload_folder="offload",
    offload_state_dict=True,
    trust_remote_code=True,
    torch_dtype=torch.float16,
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# LoRA 설정 및 적용
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [32]:
def preprocess_function(examples):
    full_texts = examples["Text"]
    tokenized = tokenizer(
        full_texts,
        truncation=True,
        padding="max_length",
        max_length=256,  # 필요에 따라 max_length를 조정
    )
    labels = tokenized["input_ids"].copy()
    labels = [
        [-100 if mask == 0 else token for mask, token in zip(attention, label)]
        for attention, label in zip(tokenized["attention_mask"], labels)
    ]
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": labels,
    }

In [33]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1405 [00:00<?, ? examples/s]

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset)

Dataset({
    features: ['Text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1254
})


In [34]:
# 학습 설정
training_args = TrainingArguments(
    output_dir="./gemma_game_recommendation",  # 모델 저장 경로
    evaluation_strategy="steps",              # 정기적으로 평가 수행
    eval_steps=20,                           # 평가 주기 (10에서 100으로 증가)
    logging_steps=20,                         # 로그 기록 주기 (10에서 50으로 증가)
    save_steps=20,                           # 모델 저장 주기 (10에서 100으로 증가)
    save_total_limit=3,                       # 저장할 체크포인트 개수 제한
    num_train_epochs=8,                       # 학습 epoch 수 (3에서 5로 증가)
    per_device_train_batch_size=4,            # 학습 배치 크기 (1에서 4로 증가)
    per_device_eval_batch_size=4,             # 평가 배치 크기 (1에서 4로 증가)
    gradient_accumulation_steps=8,
    lr_scheduler_type='linear',             # 그래디언트 누적 단계 (4에서 8로 증가)
    learning_rate=5e-4,                       # 학습률 (5e-4에서 2e-5로 감소)
    warmup_steps=100,                         # 학습률 워밍업 단계 추가
    weight_decay=0.01,
    fp16=False,  # 혼합 정밀도 비활성화
    optim="adamw_torch",  # 옵티마이저 변경
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,             # loss는 낮을수록 좋음
                     # 로그 저장 위치 (none: Colab 전용)
)


In [35]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [36]:
# 모델 학습
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
20,4.579100,4.121778
40,3.787400,3.485501
60,3.378100,3.272675
80,3.176100,3.164208
100,3.121200,3.114666
120,3.004300,3.060059
140,2.928400,3.026854
160,2.858100,2.991742
180,2.772300,2.965851
200,2.710400,2.936441


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `

TrainOutput(global_step=352, training_loss=2.9012967673215, metrics={'train_runtime': 14433.0406, 'train_samples_per_second': 0.779, 'train_steps_per_second': 0.024, 'total_flos': 3.424834314829824e+16, 'train_loss': 2.9012967673215, 'epoch': 8.0})

In [37]:
trainer.save_model('/content/drive/MyDrive/generativeai/models')
tokenizer.save_pretrained('/content/drive/MyDrive/generativeai/models')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/generativeai/models/tokenizer_config.json',
 '/content/drive/MyDrive/generativeai/models/special_tokens_map.json',
 '/content/drive/MyDrive/generativeai/models/tokenizer.model',
 '/content/drive/MyDrive/generativeai/models/added_tokens.json',
 '/content/drive/MyDrive/generativeai/models/tokenizer.json')

In [39]:
from huggingface_hub import HfApi

# Hugging Face API 객체 생성
api = HfApi()

# 모델 저장소 경로
repo_id = f"Chanjeans/gamerecommend"

# 모델 업로드
api.upload_folder(
    folder_path="/content/drive/MyDrive/generativeai/models",  # 모델 폴더 경로
    repo_id=repo_id,
    commit_message="Upload game recommendation model",
)


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/7.38M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Chanjeans/gamerecommend/commit/65271e0a5c56c589e60a69ab7527cce195daf526', commit_message='Upload game recommendation model', commit_description='', oid='65271e0a5c56c589e60a69ab7527cce195daf526', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Chanjeans/gamerecommend', endpoint='https://huggingface.co', repo_type='model', repo_id='Chanjeans/gamerecommend'), pr_revision=None, pr_num=None)

In [44]:
from math import exp
from math import exp

train_loss = 2.35  # 예: 훈련 중 기록된 Loss
val_loss = 2.84   # 예: 검증 데이터에서 기록된 Loss

train_perplexity = exp(train_loss)
val_perplexity = exp(val_loss)

print(f"Training Perplexity: {train_perplexity}")
print(f"Validation Perplexity: {val_perplexity}")


train_perplexity = exp(train_loss)
val_perplexity = exp(val_loss)

print(f"Training Perplexity: {train_perplexity}")
print(f"Validation Perplexity: {val_perplexity}")
